# 0. Install and Import Dependencies

In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

# 1. Load Model

In [2]:
interpreter = tf.lite.Interpreter(model_path='D:\Programming\ML Projects\Lib\site-packages\lite-model_movenet_singlepose_thunder_3.tflite')
interpreter.allocate_tensors()

# 2. Make Detections

# 3. Draw Keypoints

In [3]:

def calculate_angle(a,b,c):

    radians =np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0])
    angle =np.abs(radians*180.0/np.pi)
    
    if angle>180.0:
        angle = 360-angle
    
    return angle

def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 
 
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}
#knee and hip
def squat(left_shoulder,left_hip,left_knee,left_ankle):
    angle1 = calculate_angle(left_shoulder,left_hip,left_knee)
    angle2 = calculate_angle(left_hip,left_knee,left_ankle)
    cv2.putText(frame, str(angle1),[120,120],cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0),2,cv2.LINE_AA)
    cv2.putText(frame, str(angle2),[240,240],cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0),2,cv2.LINE_AA)

#armpit and elbow
def overhead_press(left_shoulder, left_elbow, left_wrist,right_shoulder, right_elbow, right_wrist, left_hip, right_hip):
    angle1 = calculate_angle(left_shoulder,left_elbow,left_wrist)
    angle2 = calculate_angle(right_shoulder, right_elbow, right_wrist)
    angle3 = calculate_angle(left_elbow, left_shoulder, left_hip)
    angle4 = calculate_angle(right_elbow, right_shoulder, right_hip)
    cv2.putText(frame, str(angle1),[120,120],cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(frame, str(angle2),[150,150],cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(frame, str(angle3),[190,190],cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(frame, str(angle4),[240,240],cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
    
#benchpress
def benchpress(left_shoulder, left_elbow, left_wrist,right_shoulder, right_elbow, right_wrist):
    angle1 = calculate_angle(left_shoulder,left_elbow,left_wrist)
    angle2 = calculate_angle(right_shoulder, right_elbow, right_wrist)
    cv2.putText(frame, str(angle1),[120,120],cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(frame, str(angle2),[150,150],cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)


#knee ,hip and back length

#back arch
def arch(nose , left_shoulder, left_hip,frame):
    angle1 = calculate_angle(nose , left_shoulder, left_hip)
    
    if(angle1 <100):
            cv2.putText(frame,"BACK BENDING",[150,140],cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)    
    cv2.putText(frame, str(angle1),[120,120],cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),2,cv2.LINE_AA)


def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)
            
def knee_bend(frame):
     cv2.putText(frame,"Knee bending too much",(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1,cv2.LINE_AA)

        
cap = cv2.VideoCapture("Overhead_press _frontview.mp4")
while cap.isOpened():
    ret, frame = cap.read()
    
    # Reshape image
    img = frame.copy() 
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 256,256)
    input_image = tf.cast(img, dtype=tf.float32)
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    
    # Rendering 
    draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4)
    
    
    left_shoulder = [keypoints_with_scores[0][0][5][0],keypoints_with_scores[0][0][5][1]]
    left_elbow = [keypoints_with_scores[0][0][7][0],keypoints_with_scores[0][0][7][1]]
    left_wrist = [keypoints_with_scores[0][0][9][0],keypoints_with_scores[0][0][9][1]]
   
        
    right_shoulder = [keypoints_with_scores[0][0][6][0],keypoints_with_scores[0][0][6][1]]
    right_elbow = [keypoints_with_scores[0][0][8][0],keypoints_with_scores[0][0][8][1]]
    right_wrist = [keypoints_with_scores[0][0][10][0],keypoints_with_scores[0][0][10][1]]

    left_hip = [keypoints_with_scores[0][0][11][0],keypoints_with_scores[0][0][11][1]]

    right_hip = [keypoints_with_scores[0][0][12][0],keypoints_with_scores[0][0][12][1]]


    overhead_press(left_shoulder, left_elbow, left_wrist,right_shoulder, right_elbow, right_wrist, left_hip, right_hip)
    cv2.rectangle(frame,(0,0),(255,73),(245,117,16),-1)

    cv2.imshow('MoveNet Thunder', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

# 4. Draw Edges